In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
#sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
#test = pd.read_csv("../input/digit-recognizer/test.csv")
train = pd.read_csv("../input/digit-recognizer/train.csv")

In [ ]:
X = np.array(train.iloc[0:41500 , 1:])
y = np.array(train.iloc[0:41500, :1]).ravel()

print(X.shape,y.shape)

In [ ]:

from keras.models import Sequential
from keras.layers import Dense , Dropout , BatchNormalization
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification




def create_network():
    network = tf.keras.Sequential()
    network.add(layers.Dense(300,activation='elu', input_shape=(784,)))
    network.add(layers.BatchNormalization())
    network.add(layers.Dropout(0.5))
    network.add(layers.Dense(100,activation='elu'))
    network.add(layers.BatchNormalization())
    network.add(layers.Dropout(0.5))
    network.add(layers.Dense(10, activation='softmax'))
    # Compile model
    network.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    network.summary()
    
    return network

neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=50, 
                                 batch_size=200, 
                                 verbose=0)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate , StratifiedShuffleSplit, ShuffleSplit
import matplotlib.pyplot as plt

#numeric_transformer=Pipeline(steps=['scaler',StandardScaler()])
scaler=StandardScaler()
classifier=MLPClassifier(hidden_layer_sizes=(300,100), activation='relu', solver='adam', alpha=0.0001, batch_size=200, 
                         learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=50, shuffle=True, 
                         random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                         nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
                         epsilon=1e-08, n_iter_no_change=10)

scikit_clf=Pipeline(steps=[('scaler',scaler),
                   ('classifier', classifier)])

tensor_clf=Pipeline(steps=[('scaler',scaler),
                   ('classifier', neural_network)])


def apply_cv(X, y, nFold, test_size, shuffled):   
    metrics = ['accuracy']
    if shuffled:
        rs = ShuffleSplit(n_splits=nFold, test_size=test_size, random_state=10)
        outcomes=cross_validate(mlp, X, y, scoring=metrics, cv=rs, return_train_score=False)
    else:
        kf = KFold(n_splits=nFold)
        outcomes=cross_validate(mlp, X, y, scoring=metrics, cv=kf, return_train_score=False)
    return outcomes['test_accuracy']






# Comparativo com dados originais

In [ ]:
n_fold=5
kf = KFold(n_splits=n_fold)


tensor_original = cross_val_score(tensor_clf, X, y, cv=kf)


scikit_original = cross_val_score(scikit_clf, X, y, cv=kf)




plt.title('Accuracy - scikit-learn & Tensorflow 41.500 instâncias')
plt.grid(color='silver', linestyle='-', linewidth=1, alpha=0.8)
plt.scatter(range(1,n_fold+1),scikit_original,color='blue',alpha=1)
plt.scatter(range(1,n_fold+1),tensor_original,color='orange',alpha=1)
plt.legend(('scikit-learn','TensorFlow'), scatterpoints =1, loc='lower left',ncol=1, fontsize=8)
plt.show()

# Os modelos estão Aprendendo?

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt
    
    



In [ ]:
estimator=scikit_clf  # pode ser qualquer estimador
plot_learning_curve(estimator, title, X_train, y_train, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

In [ ]:
estimator=tensor_clf  # pode ser qualquer estimador
plot_learning_curve(estimator, title, X_train, y_train, (0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

# Criando dados sintéticos

In [ ]:
from scipy.ndimage.interpolation import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

X_train_augmented = [image for image in X]
y_train_augmented = [label for label in y]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):   #variação de 1 pixels em todas as direções
    for image, label in zip(X, y):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train = np.array(X_train_augmented)
y_train = np.array(y_train_augmented)

X_train.shape

### Treinando com dados sintéticos

In [ ]:
tensor_augmented = cross_val_score(tensor_clf, X_train, y_train, cv=kf)
mlp_augmented = cross_val_score(scikit_clf, X_train, y_train, cv=kf)

# Gráfico geral

In [ ]:
plt.title('Accuracy - scikit-learn & Tensorflow 207.500 instâncias')
plt.grid(color='silver', linestyle='-', linewidth=1, alpha=0.8)
plt.scatter(range(1,n_fold+1),mlp_augmented,color='blue',alpha=1.0)
plt.scatter(range(1,n_fold+1),tensor_augmented,color='orange',alpha=1.0)
plt.scatter(range(1,n_fold+1),tensor_original,color='orange',alpha=0.3)
plt.scatter(range(1,n_fold+1),mlp_original,color='blue',alpha=0.3)
plt.legend(('scikit-learn','TensorFlow'), scatterpoints =1, loc='lower left',ncol=1, fontsize=8)
plt.show()